In [200]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline

In [209]:
import numpy as np
import pandas as pd
import os
import re
import shutil

In [210]:
#Labels for train and test data for both calc and mass cases

## Train data labels
calc_all_train = pd.read_csv("calc_case_description_train_set.csv")
calc_train = calc_all_train
calc_train['image_name'] = calc_all_train.patient_id + '_' + calc_all_train['left or right breast'] + '_' + calc_all_train['image view'] + '.png'
calc_train.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
calc_train.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]


In [211]:
mass_train = pd.read_csv("mass_case_description_train_set.csv")
mass_all_train = mass_train
mass_train['image_name'] = mass_all_train.patient_id + '_' + mass_all_train['left or right breast'] + '_' + mass_all_train['image view'] + '.png'
mass_train.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
mass_train.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]


In [212]:
## Test data labels
calc_test = pd.read_csv("calc_case_description_test_set.csv")
calc_all_test = calc_test
calc_test['image_name'] = calc_all_test.patient_id + '_' + calc_all_test['left or right breast'] + '_' + calc_all_test['image view'] + '.png'
calc_test.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
calc_test.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]

mass_test = pd.read_csv("mass_case_description_test_set.csv")
mass_all_test = mass_test
mass_test['image_name'] = mass_all_test.patient_id + '_' + mass_all_test['left or right breast'] + '_' + mass_all_test['image view'] + '.png'
mass_test.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
mass_test.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]

In [213]:

# Train and Test label
train_label = pd.concat([calc_all_train, mass_all_train], axis = 0)
train_label['Pathology'][train_label['Pathology'] == 'BENIGN_WITHOUT_CALLBACK'] = 'BENIGN'
train_label['Class'] = train_label['Pathology'] + '_' + train_label['Abnormality_Type']

test_label = pd.concat([calc_all_test, mass_all_test], axis = 0)
test_label['Pathology'][test_label['Pathology'] == 'BENIGN_WITHOUT_CALLBACK'] = 'BENIGN'
test_label['Class'] = test_label['Pathology'] + '_' + test_label['Abnormality_Type']


C:\Users\dahyun\anaconda3\envs\mammo\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\dahyun\anaconda3\envs\mammo\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [214]:

# Set image_name to be the index
train_label.set_index("Image_Name", inplace = True)
test_label.set_index("Image_Name", inplace = True)

#test_label.head()

In [215]:
train_label.loc['P_00034_RIGHT_CC.png']['Class']

Image_Name
P_00034_RIGHT_CC.png    MALIGNANT_calcification
P_00034_RIGHT_CC.png             MALIGNANT_mass
Name: Class, dtype: object

In [216]:
test_label.loc['P_00202_RIGHT_CC.png']['Class']

Image_Name
P_00202_RIGHT_CC.png    MALIGNANT_calcification
P_00202_RIGHT_CC.png             MALIGNANT_mass
Name: Class, dtype: object

In [201]:
#train data
CBIS_train_patches = np.load(os.path.join("npy_new3", "train_mass_patch.npy" ))
CBIS_train_labels = np.load(os.path.join("npy_new3", "train_mass_Lbl.npy" ))
CBIS_train_FNs = np.load(os.path.join("npy_new3", "train_mass_FN.npy" ))

#test data
CBIS_test_patches = np.load(os.path.join("npy_new3", "test_mass_patch.npy" ))
CBIS_test_labels = np.load(os.path.join("npy_new3", "test_mass_Lbl.npy" ))
CBIS_test_FNs = np.load(os.path.join("npy_new3", "test_mass_FN.npy" ))



print("Abnaormal train Patches:", CBIS_train_patches.shape)
print("Abnaormal train Labels:", CBIS_train_labels.shape)
print("Abnaormal train File Names:", CBIS_train_FNs.shape)
print("\n")
print("Abnaormal test Patches:", CBIS_test_patches.shape)
print("Abnaormal test Labels:", CBIS_test_labels.shape)
print("Abnaormal test File Names:", CBIS_test_FNs.shape)

Abnaormal train Patches: (1318, 256, 256, 1)
Abnaormal train Labels: (1318,)
Abnaormal train File Names: (1318,)


Abnaormal test Patches: (378, 256, 256, 1)
Abnaormal test Labels: (378,)
Abnaormal test File Names: (378,)


In [202]:
#combine train and test data 

CBIS_all_patches = np.concatenate([CBIS_train_patches, CBIS_test_patches], axis=0)
CBIS_all_labels = np.concatenate([CBIS_train_labels, CBIS_test_labels], axis=0)
CBIS_all_FNs = np.concatenate([CBIS_train_FNs, CBIS_test_FNs], axis=0)

CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs = \
shuffle(CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs, random_state=19510705)

In [233]:
CBIS_all_labels[1685][-13:]=='calcification'

True

In [259]:
diff = []
for i in range(0,CBIS_all_labels.size,1):
    print(str(i) + ' ' + CBIS_all_labels[i])
    if(CBIS_all_labels[i][-13:]=='calcification'):
        diff.append(i)

0 MALIGNANT_mass
1 BENIGN_mass
2 BENIGN_mass
3 MALIGNANT_mass
4 BENIGN_mass
5 MALIGNANT_mass
6 BENIGN_mass
7 BENIGN_mass
8 BENIGN_mass
9 MALIGNANT_mass
10 BENIGN_mass
11 BENIGN_mass
12 BENIGN_mass
13 BENIGN_mass
14 MALIGNANT_mass
15 MALIGNANT_mass
16 BENIGN_mass
17 BENIGN_mass
18 BENIGN_mass
19 MALIGNANT_mass
20 BENIGN_mass
21 BENIGN_mass
22 BENIGN_mass
23 BENIGN_mass
24 MALIGNANT_mass
25 BENIGN_mass
26 BENIGN_mass
27 MALIGNANT_mass
28 BENIGN_mass
29 BENIGN_mass
30 BENIGN_mass
31 BENIGN_mass
32 MALIGNANT_mass
33 BENIGN_mass
34 MALIGNANT_mass
35 BENIGN_mass
36 BENIGN_mass
37 BENIGN_mass
38 MALIGNANT_mass
39 MALIGNANT_mass
40 BENIGN_mass
41 MALIGNANT_mass
42 MALIGNANT_mass
43 BENIGN_mass
44 BENIGN_mass
45 MALIGNANT_mass
46 MALIGNANT_mass
47 BENIGN_mass
48 MALIGNANT_mass
49 MALIGNANT_mass
50 BENIGN_mass
51 BENIGN_mass
52 MALIGNANT_mass
53 BENIGN_mass
54 BENIGN_mass
55 MALIGNANT_mass
56 BENIGN_mass
57 BENIGN_mass
58 MALIGNANT_mass
59 BENIGN_mass
60 BENIGN_mass
61 BENIGN_mass
62 MALIGNANT_m

In [260]:
diff

[]

In [258]:
for i in diff :
    if(CBIS_all_FNs[i][5:9] == 'Test'):
        fn = CBIS_all_FNs[i][10:]
        new = test_label.loc[fn]['Class'][-1]
    else :
        fn = CBIS_all_FNs[i][14:]
        new = train_label.loc[fn]['Class'][-1]
    CBIS_all_labels[i] = new

In [182]:
CBIS_all_labels[1685] = 'MALIGNANT_mass'

In [181]:
index = 1685
print(CBIS_all_labels[index])
print(CBIS_all_FNs[index])

MALIGNANT_calcification
Mass-Training_P_00034_RIGHT_CC.png


In [261]:
#split the combined data into train and test
train_patches, test_patches, train_labels, test_labels, train_FNs, test_FNs = \
train_test_split(CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs, test_size = 0.183565, random_state=19430727)

In [262]:
#train data
train_images = train_patches
train_labels = train_labels
train_FNs = train_FNs

#test data
test_images = test_patches
test_labels = test_labels
test_FNs = test_FNs

In [263]:
test_images.shape

(312, 256, 256, 1)

In [264]:
le = preprocessing.LabelEncoder()
le.fit(train_labels)

LabelEncoder()

In [265]:
list(le.classes_)

['BENIGN_mass', 'MALIGNANT_mass']

In [266]:
#Convert Normal to 0 
train_labels_en = le.transform(train_labels)
#train_labels_en[train_labels_en==]=0

test_labels_en = le.transform(test_labels)
#test_labels_en[test_labels_en==5]=0

In [267]:
classes = le.classes_

In [268]:
train_bin_labels = np.zeros(len(train_labels_en)).astype(np.int32)
train_bin_labels[train_labels_en != 0] = 1

test_bin_labels = np.zeros(len(test_labels_en)).astype(np.int32)
test_bin_labels[test_labels_en != 0] = 1

In [269]:
train_labels_en

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

In [270]:
np.save(os.path.join("Label_mass", "train_labels_en.npy"), train_labels_en)
np.save(os.path.join("Label_mass", "test_labels_en.npy"), test_labels_en)
np.save(os.path.join("Label_mass", "train_bin_labels.npy"), train_bin_labels)
np.save(os.path.join("Label_mass", "test_bin_labels.npy"), test_bin_labels)

In [271]:
#X_val, X_test, y_val, y_test, y_val_multi, y_test_multi = \
#    train_test_split(test_images, test_bin_labels, test_labels_en, test_size=0.5, random_state=19730104)
X_train, y_train, y_train_multi = \
     shuffle(train_images, train_bin_labels, train_labels_en, random_state=100)
X_test, y_test, y_test_multi = \
     shuffle(test_images, test_bin_labels, test_labels_en, random_state=100)

In [272]:
y_test_multi

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,

In [273]:
len(X_train)

1384

In [274]:
np.save(os.path.join("Data_mass", 'X_train.npy'), X_train)
np.save(os.path.join("Data_mass", 'y_train.npy'), y_train)
np.save(os.path.join("Data_mass", 'train_labels_multi.npy'), y_train_multi)

np.save(os.path.join("Data_mass", 'X_test.npy'), X_test)
np.save(os.path.join("Data_mass", 'y_test.npy'), y_test)
np.save(os.path.join("Data_mass", 'y_test_labels_multi.npy'), y_test_multi)